# Imports

In [ ]:
import asyncio
import traceback
import warnings
from collections import defaultdict, namedtuple
from collections.abc import Mapping, Sequence
from functools import partial
from glob import glob
from importlib import reload
from numbers import Number
from operator import getitem

import cachetools
import dask
import distributed
import holoviews as hv
import hvplot.pandas
import ipywidgets as widgets
import matplotlib.pyplot as plt
import nd2reader
import numpy as np
import pandas as pd
import param
import parambokeh
import pyarrow as pa
import pyarrow.feather as feather
import pyarrow.parquet as pq
import qgrid
import scipy
import skimage.morphology
import streamz
import streamz.dataframe as sdf
import zarr
from bokeh.models.tools import HoverTool, TapTool
from cytoolz import *
from dask import delayed
from dask_jobqueue import SLURMCluster
from distributed import Client, LocalCluster, progress
from holoviews.operation.datashader import regrid
from holoviews.streams import Selection1D, Stream, param
from tqdm import tnrange, tqdm, tqdm_notebook
from traitlets import All

IDX = pd.IndexSlice

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from processing import *
# from trench_detection import *
# from trench_segmentation import *
# from trench_segmentation.watershed import *
# from util import *
# from ui import *
import common
import data_io
import diagnostics
import geometry
import image
import metadata
import trench_detection
import trench_detection.core
import trench_detection.hough
import trench_segmentation.watershed
import ui
import util
import workflow

In [ ]:
%load_ext line_profiler
hv.extension("bokeh")
%matplotlib inline
tqdm.monitor_interval = 0
asyncio.get_event_loop().set_debug(True)
import logging

logging.basicConfig(level=logging.DEBUG)

# Restore data

In [ ]:
%store -r trench_points
%store -r trench_diag
%store -r trench_bboxes
trench_bboxes_t0 = util.get_one(trench_bboxes.groupby("t"))[1]

# Config

In [ ]:
client = Client(ip="0.0.0.0", n_workers=1, threads_per_worker=1, processes=False)

In [ ]:
dask.config.config["distributed"]

In [ ]:
dask.config.config["distributed"]["worker"]["profile"] = {
    "interval": "10s",
    "cycle": "10s",
}
# {'interval': '10ms', 'cycle': '1000ms'}

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="09:00:00",
    # job_extra=['-p transfer'],
    # job_extra=['--cores-per-socket=8'],
    # job_extra=['--exclude=compute-e-16-181,compute-e-16-186'],
    # interface='ib0',
    memory="1GB",
    local_directory="/tmp",
    cores=1,
    processes=1,
    # diagnostics_port=('127.0.0.1', 8787),
    env_extra=[
        'export PYTHONPATH="/home/jqs1/projects/matriarch"',
        #'export PYTHONTRACEMALLOC=25',
        #'export MALLOC_CONF=prof:true,prof_leak:true,lg_prof_interval:31,prof_final:true',
        'export LD_PRELOAD="/home/jqs1/lib/libjemalloc.so.2"',
    ],
)
client = Client(cluster)  # , direct_to_workers=True)

In [ ]:
cluster.scale(400)

In [ ]:
cluster._widget().children[1].children[1].children[0].children[0].layout.width = "200px"
cluster

In [ ]:
# cluster = LocalCluster(n_workers=1)
# client = Client(cluster)

In [ ]:
# cluster.adapt(minimum=10,maximum=200)

In [ ]:
cluster.stop_jobs(cluster.running_jobs.keys())

In [ ]:
cluster.scheduler.stop_services()
cluster.scheduler.stop()

In [ ]:
client.restart()

In [ ]:
del cluster

In [ ]:
# client = Client()

In [ ]:
reload(distributed)
from distributed import Client

In [ ]:
client = Client(ip="0.0.0.0", n_workers=1, threads_per_worker=1, processes=False)

In [ ]:
client = Client(cluster)

# Loading data

In [ ]:
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/180405_txnerr.nd2', '/n/scratch2/jqs1/fidelity/all/180405_txnerr001.nd2']
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2']#, '/n/scratch2/jqs1/fidelity/all/TrErr002_Exp.nd2']
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/TrErr002_Exp.nd2']
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/180405_txnerr.nd2', '/n/scratch2/jqs1/fidelity/all/180405_txnerr001.nd2',
#                 '/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2', '/n/scratch2/jqs1/fidelity/all/TrErr002_Exp.nd2']
# nd2_filenames = ['/home/jqs1/scratch/fidelity/180518_triplegrowthcurve/PHASE_GC001.nd2', '/home/jqs1/scratch/fidelity/180518_triplegrowthcurve/PHASE_GC002.nd2']
nd2_filenames = glob("/n/scratch2/jqs1/fidelity/all/180405_*.nd2") + glob(
    "/n/scratch2/jqs1/fidelity/all/TrErr*.nd2"
)

In [ ]:
all_frames, metadata, parsed_metadata = workflow.get_nd2_frame_list(nd2_filenames)
image_limits = workflow.get_filename_image_limits(metadata)

# Reload

In [ ]:
def _check():
    import os

    return os.path.exists(
        "/n/scratch2/jqs1/fidelity/all/180405_txnerr_loweronly_fast.nd2"
    )


[k for k, v in client.run(_check).items() if v is False]

In [ ]:
def do_reload():
    from importlib import reload

    import diagnostics
    import image
    import trench_detection
    import util
    import workflow

    # reload(util)
    # reload(trench_detection.hough)
    # reload(diagnostics)
    reload(workflow)
    # reload(image)


client.run(do_reload)
do_reload()

# Finding trenches

In [ ]:
frames_to_process = all_frames.loc[IDX[:, :, ["MCHERRY"], 0], :]

In [ ]:
len(frames_to_process)

## Run trench finding

In [ ]:
%%time
find_trenches_diag = diagnostics.wrap_diagnostics(
    trench_detection.hough.find_trenches, ignore_exceptions=True, pandas=True
)
trench_info_futures = {
    idx: client.submit(
        find_trenches_diag, client.submit(workflow.get_nd2_frame, **idx._asdict())
    )
    for idx, row in util.iter_index(frames_to_process)
}

In [ ]:
progress(trench_info_futures)

In [ ]:
client.cancel(trench_info_futures)

In [ ]:
def as_completed(obj, with_results=True):
    if isinstance(obj, Mapping):
        futures = obj.values()
        dask_to_keys = {future.key: k for k, future in obj.items()}
    else:
        raise NotImplementedError
    for res in distributed.as_completed(futures, with_results=with_results):
        if with_results:
            future, result = res
            yield dask_to_keys[future.key], future, result
        else:
            future = res
            yield dask_to_keys[future.key], future

In [ ]:
trench_info = {}
for key, fut, res in as_completed(trench_info_futures):
    trench_info[key] = res
    client.cancel(fut)

In [ ]:
%%time
trench_info = util.apply_map_futures(
    client.gather, trench_info_futures, predicate=lambda x: x.status == "finished"
)

In [ ]:
%%time
trench_points, trench_diag, trench_err = workflow.unzip_trench_info(trench_info)

In [ ]:
len(trench_points)

In [ ]:
%%time
%store trench_points
%store trench_diag

## Analysis

In [ ]:
bad_angle = trench_diag["find_trench_lines.hough_2.angle"].abs() > 2
bad_angle.sum()

In [ ]:
bad_pitch = (trench_diag["find_trench_lines.hough_2.peak_func.pitch"] - 24).abs() > 1
bad_pitch.sum()

In [ ]:
selected = trench_diag[bad_pitch]  # trench_diag[bad_angle | bad_period]

In [ ]:
frame_stream.event(_df=selected.index.to_frame(index=False))

In [ ]:
%%time
trench_points_good = trench_points[~util.multi_join(trench_points.index, bad_pitch)]

In [ ]:
(len(trench_points_good), len(trench_points_good) / len(trench_points))

In [ ]:
%%time
trench_bbox_futures = []
for _, trenches in trench_points_good.groupby(["filename", "position", "t"]):
    trench_bbox_futures.append(
        client.submit(workflow.get_trench_bboxes, trenches, image_limits)
    )

In [ ]:
%%time
trench_bbox_results = util.apply_map_futures(
    client.gather, trench_bbox_futures, predicate=lambda x: x.status == "finished"
)
trench_bboxes = pd.concat(
    [trench_points_good, pd.concat(trench_bbox_results, axis=0)], axis=1
)

In [ ]:
%%time
%store trench_bboxes

In [ ]:
%store -r trench_bboxes

In [ ]:
trench_bboxes_t0 = util.get_one(trench_bboxes.groupby("t"))[1]
# trench_bboxes_t0.index = trench_points_good_t0.index.droplevel('t')

# Trench finding QA

In [ ]:
selected = all_frames

In [ ]:
FrameStream = ui.DataFrameStream.define(
    "FrameStream", selected.index.to_frame(index=False)
)
frame_stream = FrameStream()

box = ui.dataframe_browser(frame_stream)
frame_stream.event()
box

In [ ]:
ui.image_viewer(frame_stream)

In [ ]:
ui.show_frame_info(trench_diag, frame_stream)

In [ ]:
g = ui.show_grid(selected, stream=frame_stream)
g

In [ ]:
frame = workflow.get_nd2_frame_anyargs(**dict(frame_stream.get_param_values()))

In [ ]:
tp, diag, _ = diagnostics.wrap_diagnostics(
    trench_detection.hough.find_trenches, ignore_exceptions=False
)(frame)

In [ ]:
ui.show_plot_browser(diag)

# Segmentation

In [ ]:
selected_trenches_segmentation = trench_bboxes_t0[
    trench_bboxes_t0[("info", "hough_value")] > 90
].loc[IDX[:, :, ["MCHERRY"], 0, :, :], :]

In [ ]:
(len(trench_bboxes_t0), len(selected_trenches_segmentation) / len(trench_bboxes_t0))

In [ ]:
# frames_to_analyze = all_frames.loc[IDX[:,:1,['MCHERRY','YFP'],1:5],:]
frames_to_analyze = all_frames.loc[IDX[:, :, ["MCHERRY", "YFP"], :], :]

In [ ]:
(
    len(frames_to_analyze),
    len(all_frames.loc[IDX[:, :, ["MCHERRY", "YFP"], :], :]) / len(frames_to_analyze),
)

In [ ]:
labelwise_funcs = {
    "mean": np.mean,
    "min": np.min,
    "max": np.max,
    ("p0.3", "p0.5", "p0.7", "p0.9", "p0.95"): partial(
        np.percentile, q=(30, 50, 70, 90, 95)
    ),
}
trenchwise_funcs = {"sharpness": image.sharpness, **labelwise_funcs}
framewise_funcs = {"sharpness": image.sharpness, **labelwise_funcs}

analyze_trench_func = partial(
    workflow.analyze_trenches,
    framewise_funcs=framewise_funcs,
    trenchwise_funcs=trenchwise_funcs,
    labelwise_funcs=labelwise_funcs,
    regionprops=True,
    segment_func=trench_segmentation.watershed.segment_trench,
)
analyze_trench_func = compose(
    list, partial(map, pa.RecordBatch.from_pandas), analyze_trench_func
)

# analyze_trench_func = compose(lambda x: None, analyze_trench_func)

analyze_trench_func = partial(client.submit, analyze_trench_func, retries=2)

analysis_futures_iter = workflow.analyze_frames_and_trenches_iter(
    selected_trenches_segmentation, frames_to_analyze, analyze_trench_func
)

# analysis_futures = workflow.analyze_frames_and_trenches(selected_trenches_segmentation,
#                                            frames_to_analyze,
#                                            analyze_trench_func)

# display(trenchwise_df)

## Streaming gather

In [ ]:
%%time
ac = distributed.as_completed([], with_results=False, loop=client.loop)

new_futures_stream = streamz.Stream()
finished_futures_stream = streamz.Stream(asynchronous=True, loop=client.loop)

stream_sinks = {}
stream_writers = {}
output_filename = "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_{}.arrow"

new_futures_stream.sink(lambda x: ac.add(x))

errored_futures = set()
finished_futures_stream.filter(lambda x: x.status == "error").sink(
    lambda x: errored_futures.add(x)
)


def timeout_func(futures):
    ac.update(futures)


successful_futures_stream = finished_futures_stream.filter(
    lambda x: x.status == "finished"
)
# batched_futures_stream = successful_futures_stream.rate_limit(0.0004).timed_window(1)
# gathered_futures_stream = streamz.buffer(batched_futures_stream, 10).gather_and_cancel(client=client, cancel=True)
# batched_futures_stream = successful_futures_stream.timed_window(1)
batched_futures_stream = successful_futures_stream.rate_limit(0.01).timed_window(5)
buffered_futures_stream = (
    batched_futures_stream  # streamz.buffer(batched_futures_stream, 10)
)

cancelled_futures = set()
gathered_futures_stream = buffered_futures_stream.gather_and_cancel(
    client=client,
    gather=True,
    cancel=True,
    timeout=4,
    timeout_func=timeout_func,
    success_func=cancelled_futures.update,
)
# gathered_futures_stream.flatten().sink(partial(workflow.sink_to_arrow, sinks=stream_sinks, writers=stream_writers, output_func=lambda i: pa.OSFile(output_filename.format(i), 'w')))
write_failures = []
flattened_futures_stream = gathered_futures_stream.flatten()
writer_stream = (
    flattened_futures_stream  # .timed_window(10).map(lambda x: list(zip(*x)))
)
sink_func = partial(
    workflow.sink_to_arrow,
    sinks=stream_sinks,
    writers=stream_writers,
    output_func=lambda i: pa.OSFile(output_filename.format(i), "w"),
)
# sink_func = partial(client.loop.run_in_executor, None, sink_func)
# writer_stream.with_timeout(timeout=3, retries=2, failure_func=write_failures.append).sink(sink_func)
writer_stream.sink(sink_func)
# stored_data = writer_stream.sink_to_list()

# finished_futures_stream.sink(excepts(StopIteration, lambda x: new_futures_stream.emit(next(analysis_futures_iter)) if should_add_task()))
# new_futures_stream.sink_to_list()
all_futures = set()
finished_futures = set()
new_futures_stream.sink(lambda x: all_futures.add(x))
successful_futures_stream.sink(lambda x: finished_futures.add(x))

TASK_BUFFER_SIZE = 10000

# def should_add_task():
#     #return len([f for f in all_futures if f.status == 'pending'])
#     #return TASK_BUFFER_SIZE > len(all_futures - finished_futures)
#     return TASK_BUFFER_SIZE > len(all_futures - cancelled_futures)
#     #print('>',len(all_futures - finished_futures))
#     #return True

# def readd_task(x):
#     if should_add_task():
#         return new_futures_stream.emit(next(analysis_futures_iter))


def readd_task(x):
    num_tasks_needed = TASK_BUFFER_SIZE - len(
        all_futures - cancelled_futures - errored_futures
    )
    if num_tasks_needed > 0:
        for future in take(num_tasks_needed, analysis_futures_iter):
            new_futures_stream.emit(future)


finished_futures_stream.sink(excepts(StopIteration, readd_task))

# ac.update(take(3000, analysis_futures_iter))
for future in take(TASK_BUFFER_SIZE, analysis_futures_iter):
    new_futures_stream.emit(future)

gather_func = workflow.gather_stream(finished_futures_stream, ac)
gather_task = client.loop.asyncio_loop.create_task(gather_func)

In [ ]:
ac.count()

In [ ]:
gather_task

In [ ]:
for future in take(5000, analysis_futures_iter):
    new_futures_stream.emit(future)
gather_func = workflow.gather_stream(finished_futures_stream, ac)
gather_task = client.loop.asyncio_loop.create_task(gather_func)

In [ ]:
gather_task.cancel()
client.cancel(all_futures)

In [ ]:
cluster.scale(200)

In [ ]:
len(all_futures)

In [ ]:
errored_futures[-1].result()

In [ ]:
%%time
pa.open_stream(stream_sinks[2].r()).read_pandas()

In [ ]:
util.apply_map_futures(
    client.gather, analysis_futures, predicate=lambda x: x.status == "error"
)

# Analysis

## Load data

In [ ]:
%%time
framewise_df = data_io.read_parquet(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_0.parquet4"
).to_pandas()

In [ ]:
%%time
trenchwise_df = data_io.read_parquet(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_1.parquet4"
).to_pandas()

In [ ]:
trenchwise_df.columns = ["/".join(col).strip() for col in trenchwise_df.columns.values]

In [ ]:
cols = [
    "filename",
    "position",
    "channel",
    "t",
    "trench_set",
    "trench",
    "label",
    "('YFP', 'labelwise', 'p0.9')",
    "('MCHERRY', 'labelwise', 'p0.9')",
    "('YFP', 'regionprops', 'area')",
]

In [ ]:
%%time
labelwise_df = data_io.read_parquet(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.sorted3.parquet4",
    columns=cols,
).to_pandas()

In [ ]:
# TODO: otherwise computing is_unique is costly when we want to get_loc with full key
labelwise_df.index.__dict__["_cache"] = {"lexsort_depth": 6, "is_unique": True}
# labelwise_df.index.lexsort_depth # prime the cache
# if '_cache' not in labelwise_df.index.__dict__:
#     labelwise_df.index.__dict__['_cache'] = {}
# labelwise_df.index._cache['is_unique'] = True

In [ ]:
labelwise_df.columns = ["/".join(col).strip() for col in labelwise_df.columns.values]

In [ ]:
%%time
labelwise_df.index.lexsort_depth

In [ ]:
%%time
labelwise_df.index.is_unique

In [ ]:
def func(data):
    data = data.loc[IDX[:, :, 10:], :]
    # cell_sized = data[(data[area] > 100) & (data[area] < 200)]
    cell_sized = data[data[area].between(100, 200)]
    # median = cell_sized[yfp].median()
    bright_ts = (cell_sized[cell_sized[yfp] > 200][yfp].groupby("t").count() > 2).sum()
    # num_brights = (cell_sized & (data[yfp] > 1.2*median)).sum()
    # return (median < 200) and (100 > bright_ts > 1)
    return bright_ts


%lprun -f func labelwise_subset.groupby(trench_key).apply(func)

In [ ]:
a = labelwise_subset.groupby(trench_key).apply(func)

In [ ]:
a.to_frame()

## Burst detection

In [ ]:
yfp = "YFP/labelwise/p0.9"
mcherry = "MCHERRY/labelwise/p0.9"
area = "YFP/regionprops/area"
trench_key = ["filename", "position", "trench_set", "trench"]
trench_t_key = ["filename", "position", "trench_set", "trench", "t"]

In [ ]:
%%time
# labelwise_subset = labelwise_df.loc[IDX['/n/scratch2/jqs1/fidelity/all/TrErr002_Exp.nd2',:],:]
# labelwise_subset = labelwise_df.loc[IDX['/n/scratch2/jqs1/fidelity/all/180405_txnerr_loweronly_fast.nd2',:],:]
labelwise_subset = labelwise_df.loc[
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr_loweronly_fast.nd2", :], :
]

In [ ]:
%%time
labelwise_selected = labelwise_subset  # [(labelwise_subset[area] > 100) & (labelwise_subset[area] < 400)]

In [ ]:
%%time
labelwise_selected2 = util.multi_join(labelwise_selected, trenchwise_df["YFP/p0.3"])

In [ ]:
%%time
background = labelwise_selected2.loc[IDX[:, :, :, :, :, 0], [yfp, mcherry]]
background.index = background.index.droplevel("label")
background.columns = [c + "_bg" for c in background.columns]
cell_sized = labelwise_selected2[labelwise_selected2[area].between(100, 200)].loc[
    IDX[:, :, :, :, :, 1:], :
]
cell_sized = util.multi_join(cell_sized, background)

In [ ]:
%%time


def func(data):
    bright_ts = (
        data[data[yfp] - data[yfp + "_bg"] > 10][yfp].groupby("t").count() > 2
    ).sum()
    # bright_ts = data[data[yfp] - data[yfp+'_bg'] > 10][yfp].groupby('t')
    return bright_ts


bright_ts = cell_sized.groupby(trench_key).apply(func)
bright_ts.name = "bright_ts"

In [ ]:
bright_ts.sum()

In [ ]:
(bright_ts != 0).sum() / (bright_ts == 0).sum()

In [ ]:
plt.hist(bright_ts, bins=50, log=True, normed=True)

In [ ]:
len(bright_ts)

In [ ]:
%%time
labelwise_selected3 = util.multi_join(labelwise_selected2, bright_ts)

In [ ]:
%%time
detected_bursts = labelwise_selected3[labelwise_selected3["bright_ts"] == 1]

In [ ]:
%%time
len(detected_bursts.groupby(trench_key))

## New visualization

In [ ]:
LabelStream = ui.MultiIndexStream.define("LabelStream", labelwise_df.index)
label_stream = LabelStream()
box = ui.dataframe_browser(label_stream)
label_stream.event()
box

In [ ]:
%%opts Layout [normalize=False]
%%output size=100
hover = HoverTool(
    tooltips=[
        ("(x,y)", "(@x{0[.]0}, @y{0[.]0})"),
        ("value", "@z"),
    ]
)
# cb = compose(partial(ui.hover_image, hover), ui._trench_img, workflow.get_trench_image)
cb = lambda v_max: compose(
    partial(ui.hover_image, hover),
    lambda x: x.redim.range(z=(0, v_max)),
    ui._trench_img,
    workflow.get_trench_image,
)
# cb = workflow.get_trench_image
(
    ui.trench_viewer(
        trench_bboxes, label_stream, channel="MCHERRY", image_callback=cb(4000)
    )
    + ui.trench_viewer(
        trench_bboxes, label_stream, channel="YFP", image_callback=cb(800)
    )
).cols(1)

In [ ]:
groups = detected_bursts.groupby(trench_key)
group_set_keys = list(util.grouper(groups.groups.keys(), 10))
group_index = pd.MultiIndex.from_tuples([(i,) for i in range(len(group_set_keys))])
group_index.names = ["group_set"]

In [ ]:
GroupStream = ui.MultiIndexStream.define("GroupStream", group_index)
group_stream = GroupStream()
group_box = ui.dataframe_browser(group_stream)
group_stream.event()
group_box

In [ ]:
%%output size=180
sel = Selection1D()


def callback(group_set):
    df = pd.concat([groups.get_group(key) for key in group_set_keys[group_set]])
    plot = hv.Scatter(
        df,
        kdims=["t"],
        vdims=["YFP/labelwise/p0.9", "filename", "position", "trench_set", "trench"],
    )
    tooltips = [
        ("t", "@t{0[.]0}"),
        # ('filename', '@filename'),
        ("trench", "@position.@trench_set.@trench"),
        ("YFP", "@{YFP/labelwise/p0.9}{0[.]0}"),
    ]
    hover = HoverTool(tooltips=tooltips)
    tap = TapTool()
    plot = plot.options(
        "Scatter",
        size=3,
        color_index="trench",
        nonselection_alpha=0.3,
        cmap="Category20",
        tools=[hover, tap],
        show_legend=True,
    )
    # ui.selection_to_stream(plot, label_stream)
    sel.clear()
    sel.add_subscriber(
        partial(
            ui._selection_to_stream_callback,
            data=plot.data,
            keys=df.index.names,
            stream=label_stream,
        )
    )
    return plot


p = hv.DynamicMap(callback, streams=[group_stream])
sel.source = p
p

## Old visualization

In [ ]:
%%output size=180
# p = hv.Scatter(detected_bursts, kdims=['t'], vdims=['YFP/labelwise/p0.9', 'filename', 'position', 'trench_set', 'trench'])
d = {
    i: hv.Scatter(
        pd.concat([g[1] for g in groups]),
        kdims=["t"],
        vdims=["YFP/labelwise/p0.9", "filename", "position", "trench_set", "trench"],
    )
    for i, groups in enumerate(util.grouper(detected_bursts.groupby(trench_key), 5))
}
p = hv.HoloMap(d)
tooltips = [
    ("t", "@t{0[.]0}"),
    # ('filename', '@filename'),
    ("trench", "@position.@trench_set.@trench"),
    ("YFP", "@{YFP/labelwise/p0.9}{0[.]0}"),
]
hover = HoverTool(tooltips=tooltips)
tap = TapTool()
p = p.options(
    "Scatter",
    size=3,
    color_index="trench",
    nonselection_alpha=0.3,
    cmap="Category20",
    tools=[hover, tap],
)
# pr = hv.renderer('bokeh').get_plot(p).state
# renderer = pr.renderers[-1]
# renderer.nonselection_glyph = renderer.glyph
p

In [ ]:
%%output size=180
hv.Scatter(
    detected_bursts[:10_000], kdims=["t"], vdims=["YFP/labelwise/p0.9", "trench"]
).options(size=3, color_index="trench", cmap="Category20", tools=["hover", "tap"])

In [ ]:
trenchwise_df.index.names

In [ ]:
"{:.2n}".format(1.12345)

In [ ]:
ui.dataframe_viewer(
    partial(ui.select_dataframe, labelwise_df), label_stream, label=slice(None)
)

In [ ]:
%%output size=180
p = hv.Scatter(
    detected_bursts[10_000:20_000], kdims=["t"], vdims=["YFP/labelwise/p0.9", "trench"]
).options(
    size=3,
    color_index="trench",
    nonselection_alpha=0.3,
    tools=["hover", "tap"],
    cmap="Category20",
)
ui.selection_to_stream(p, label_stream)
p

In [ ]:
def cb(**kwargs):
    dat = labelwise_df.loc[
        workflow.stream_slice(
            labelwise_df.index.names, kwargs, t=slice(None), label=slice(None)
        ),
        :,
    ]
    plot = dat.hvplot.scatter("t", "YFP_labelwise_p0.95").options(
        tools=["tap", "hover"]
    )
    # ui.selection_to_stream(plot, label_stream)
    return plot


ui.viewer(cb, label_stream)

## Grouped trench scatterplot UI

In [ ]:
groups = detected_bursts.groupby(trench_key)
group_set_keys = list(util.grouper(groups.groups.keys(), 10))
group_index = pd.MultiIndex.from_tuples([(i,) for i in range(len(group_set_keys))])
group_index.names = ["group_set"]

In [ ]:
GroupStream = ui.MultiIndexStream.define("GroupStream", group_index)
group_stream = GroupStream()
group_box = ui.dataframe_browser(group_stream)
group_stream.event()
group_box

In [ ]:
%%output size=180
sel = Selection1D()


def callback(group_set):
    df = pd.concat([groups.get_group(key) for key in group_set_keys[group_set]])
    plot = hv.Scatter(
        df,
        kdims=["t"],
        vdims=["YFP/labelwise/p0.9", "filename", "position", "trench_set", "trench"],
    )
    tooltips = [
        ("t", "@t{0[.]0}"),
        # ('filename', '@filename'),
        ("trench", "@position.@trench_set.@trench"),
        ("YFP", "@{YFP/labelwise/p0.9}{0[.]0}"),
    ]
    hover = HoverTool(tooltips=tooltips)
    tap = TapTool()
    plot = plot.options(
        "Scatter",
        size=3,
        color_index="trench",
        nonselection_alpha=0.3,
        cmap="Category20",
        tools=[hover, tap],
        show_legend=True,
    )
    # ui.selection_to_stream(plot, label_stream)
    sel.clear()
    sel.add_subscriber(
        partial(
            ui._selection_to_stream_callback,
            data=plot.data,
            keys=df.index.names,
            stream=label_stream,
        )
    )
    return plot


p = hv.DynamicMap(callback, streams=[group_stream])
sel.source = p
p

In [ ]:
%prun group_stream.event(group_set=3)

In [ ]:
sel

## Grouped trench movie UI

In [ ]:
group_box

In [ ]:
%%output size=180
# sel = Selection1D()
def callback(group_set):
    #     df = pd.concat([groups.get_group(key) for key in group_set_keys[group_set]])
    #     plot = hv.Scatter(df,
    #                       kdims=['t'],
    #                       vdims=['YFP/labelwise/p0.9', 'filename',
    #                              'position', 'trench_set', 'trench'])
    #     tooltips = [('t', '@t{0[.]0}'),
    #             #('filename', '@filename'),
    #             ('trench', '@position.@trench_set.@trench'),
    #             ('YFP', '@{YFP/labelwise/p0.9}{0[.]0}')]
    #     hover = HoverTool(tooltips=tooltips)
    #     tap = TapTool()
    #     plot = plot.options('Scatter',
    #                         size=3,
    #                         color_index='trench',
    #                         nonselection_alpha=0.3,
    #                         cmap='Category20',
    #                         tools=[hover, tap],
    #                         show_legend=True)
    # ui.selection_to_stream(plot, label_stream)
    #     sel.clear()
    #     sel.add_subscriber(partial(ui._selection_to_stream_callback,
    #                                data=plot.data,
    #                                keys=df.index.names,
    #                                stream=label_stream))
    movies = []
    for key in group_set_keys[group_set]:
        ts = groups.get_group(key).index.get_level_values("t")
        movie = hv.HoloMap(
            {
                t: ui._trench_img(
                    workflow.get_trench_image(
                        trench_bboxes, key[0], key[1], "YFP", t, key[2], key[3]
                    )
                )
                for t in ts
            }
        )
        movies.append(movie)
    plot = hv.Layout(movies).cols(1)
    return plot


p = hv.DynamicMap(callback, streams=[group_stream])
# sel.source = p
p

## Basic stats

In [ ]:
# labelwise_df2 = labelwise_df.copy()
labelwise_df.columns = ["/".join(col).strip() for col in labelwise_df.columns.values]

In [ ]:
labelwise_df.loc[:, "MCHERRY/regionprops/area"].plot(kind="hist", bins=100, log=False)

In [ ]:
selected_cells = labelwise_df2[
    (50 < labelwise_df2["MCHERRY/regionprops/area"])
    & (labelwise_df2["MCHERRY/regionprops/area"] < 300)
]

In [ ]:
selected_cells.loc[:, "MCHERRY/regionprops/area"].plot(kind="hist", bins=100, log=False)

In [ ]:
selected_cells.loc[:, "MCHERRY/labelwise/p0.5"].plot(kind="hist", bins=100, log=False)

In [ ]:
selected_cells.loc[:, "YFP/labelwise/p0.5"].plot(kind="hist", bins=100, log=True)

In [ ]:
labelwise_df[:10000].hvplot(
    x="MCHERRY/labelwise/p0.5", y="YFP/labelwise/p0.5", kind="scatter"
)

In [ ]:
yfp_cols = [c for c in labelwise_df.columns if c.startswith("YFP/labelwise/p")]
mcherry_cols = [c for c in labelwise_df.columns if c.startswith("MCHERRY/labelwise/p")]

In [ ]:
d = labelwise_df.loc[
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 2, :, 1, 50:60], yfp_cols
]

In [ ]:
d.hvplot.scatter("t", "YFP/labelwise/p0.95", by="trench")

In [ ]:
%%opts Scatter (size=3)
hv.Layout(
    [
        d.hvplot.scatter("t", c, by="trench", width=500, height=300)
        for c in reversed(yfp_cols)
    ]
).cols(1)

In [ ]:
d2 = labelwise_df.loc[
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 3, :, :, :], yfp_cols
]

In [ ]:
%%opts Scatter (size=3) [show_legend=False]
hv.Layout(
    [
        d2.hvplot.scatter("t", c, by="trench", width=500, height=300)
        for c in reversed(yfp_cols)
    ]
).cols(1)

In [ ]:
%%opts Scatter (size=3) [show_legend=False]
def get_plot(pos):
    c = "YFP/labelwise/p0.9"
    d = labelwise_df.loc[
        IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", pos, :, :, :], c
    ]
    return d.hvplot.scatter("t", c, by="trench", width=500, height=300)


hv.HoloMap(
    {
        pos: get_plot(pos)
        for pos in labelwise_df.index._get_level_values(1, unique=True)[:20]
    }
)

In [ ]:
labelwise_df.head().loc[:, cols]

In [ ]:
d = labelwise_df.loc[
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 2, :, 1, 50:60], cols
]
m = pd.melt(d.loc[:, cols])
# m.index = d.index
# m

# Trench UI

In [ ]:
trenchwise_df.head(10)

In [ ]:
trench_bboxes.head(20)

In [ ]:
trench_bboxes[trench_bboxes[("info", "hough_value")] > 90].loc[
    IDX[:, :, ["MCHERRY"], 0, :, :], :
].head(10)

In [ ]:
trench_bboxes.head(10)

In [ ]:
selected_trenches_segmentation.head(10)

In [ ]:
# LabelStream = ui.MultiIndexStream.define('LabelStream', trench_bboxes.index)
LabelStream = ui.MultiIndexStream.define("LabelStream", labelwise_df.index)
# LabelStream = ui.MultiIndexStream.define('LabelStream', selected_trenches_segmentation.index)
label_stream = LabelStream()

box = ui.dataframe_browser(label_stream)
label_stream.event()
box

In [ ]:
label_stream.contents

In [ ]:
%%opts Layout [normalize=False]
%%opts Image [width=500 height=300]
# image_callback = partial(ui.get_trench_set_overlay, get_frame_func=workflow.get_nd2_frame_cached)
# p = ui.trench_set_viewer(trench_bboxes_t0, label_stream, channel='MCHERRY', image_callback=image_callback).options({'Bounds': dict(tools=['tap'])})
p = ui.get_trench_set_overlay(
    trench_bboxes_t0,
    channel="MCHERRY",
    **{
        k: label_stream.contents[k]
        for k in ("filename", "position", "t", "trench_set", "trench")
    },
)
p = p.options({"Bounds": dict(tools=["tap"])})
s = Selection1D(source=p)
p

In [ ]:
s

In [ ]:
s

In [ ]:
%%opts Layout [normalize=False]
%%opts Image [width=800 height=150]
image_callback = partial(
    ui.get_trench_set_overlay, get_frame_func=workflow.get_nd2_frame_cached
)
(
    ui.trench_set_viewer(
        trench_bboxes_t0, label_stream, channel="MCHERRY", image_callback=image_callback
    )
    + ui.trench_set_viewer(
        trench_bboxes_t0, label_stream, channel="YFP", image_callback=image_callback
    )
).cols(1)

In [ ]:
%%opts Layout [normalize=False]
%%output size=100
hover = HoverTool(
    tooltips=[
        ("(x,y)", "(@x{0[.]0}, @y{0[.]0})"),
        ("value", "@z"),
    ]
)
cb = compose(partial(ui.hover_image, hover), ui._trench_img, workflow.get_trench_image)
# cb = workflow.get_trench_image
(
    ui.trench_viewer(trench_bboxes, label_stream, channel="MCHERRY", image_callback=cb)
    + ui.trench_viewer(trench_bboxes, label_stream, channel="YFP", image_callback=cb)
).cols(1)

In [ ]:
import bokeh

In [ ]:
compose(partial(ui.hover_image, hover), ui._trench_img, workflow.get_trench_image)(
    trench_bboxes,
    "/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2",
    3,
    "MCHERRY",
    0,
    1,
    56,
)

In [ ]:
yfp = labelwise_df.loc[
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 3, 3, 1, 61],
    ("YFP", "labelwise", "mean"),
]

In [ ]:
yfp.plot.hist(bins=50)

In [ ]:
yfp2 = labelwise_df.loc[
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 3],
    ("YFP", "labelwise", "p0.95"),
]

In [ ]:
yfp2[yfp2 < 300].plot.hist(bins=50, log=True)

In [ ]:
yfp2[yfp2 > 200]

In [ ]:
yfp2.loc[IDX[:, 1, 60]]

In [ ]:
yfp2.swaplevel(0, 1).swaplevel(1, 2).sort_index()

In [ ]:
yfp2[yfp2 > 130].groupby(["trench_set", "trench"]).filter(lambda x: x.count() > 10)

In [ ]:
yfp[yfp > 130]

In [ ]:
labelwise_df.xs(
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 3, 2, 1, 56],
    drop_level=False,
)["YFP"]

In [ ]:
labelwise_df.loc[
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 0, 2, 1, 56], "MCHERRY"
]

In [ ]:
plt.imshow(
    workflow.get_trench_image(
        trench_bboxes,
        channel="MCHERRY",
        **dissoc(label_stream.contents, "_df", "label"),
    )
)

In [ ]:
workflow.get_nd2_frame(
    channel="MCHERRY",
    **util.get_keys(label_stream.contents, "filename", "position", "t"),
)

In [ ]:
f = partial(util.get_keys, keys=["label"])
f(label_stream.contents)

In [ ]:
label_stream.contents.keys()

In [ ]:
ui.image_viewer(frame_stream)

In [ ]:
ui.show_frame_info(trench_diag, frame_stream)

In [ ]:
g = ui.show_grid(selected, stream=frame_stream)
g